In [1]:
# Copyright 2020 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# HugeCTR demo on Movie lens data

## Overview

HugeCTR is a recommender specific framework which is capable of distributed training across multiple GPUs and nodes for Click-Through-Rate (CTR) estimation. It is a component of NVIDIA [Merlin](https://developer.nvidia.com/nvidia-merlin#getstarted), which is a framework accelerating the entire pipeline from data ingestion and training to deploying GPU-accelerated recommender systems.

### Learning objectives

This notebook demonstrates the steps for training a deep learning recommender model (DLRM) on the movie lens 20M [dataset](https://grouplens.org/datasets/movielens/20m/). We will walk you through the process of data preprocessing, train a DLRM model with HugeCTR, then using the movie embedding to answer item similarity queries.

## Content
1. [Pre-requisite](#1)
1. [Data download and preprocessing](#2)
1. [HugeCTR DLRM training](#3)
1. [Answer item similarity with DLRM embedding](#4)


<a id="1"></a>
## 1. Pre-requisite


### 1.1 Docker containers
Follow the steps in [README](README.md) to build and start a HugeCTR development Docker container for the experiments.

### 1.2 Hardware
This notebook requires a Pascal, Volta, Turing, Ampere or newer GPUs, such as P100, V100, T4 or A100. 

In [2]:
!nvidia-smi

Mon Aug 24 15:34:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.24       Driver Version: 450.24       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:1A:00.0 Off |                  Off |
| N/A   27C    P8     9W /  70W |      0MiB / 16127MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            On   | 00000000:1B:00.0 Off |                  Off |
| N/A   

<a id="2"></a>
## 2. Data download and preprocessing

We first install a few extra utilities for data preprocessing.

In [3]:
!pip3 install torch tqdm
!apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


Next, we download and unzip the movie lens 20M [dataset](https://grouplens.org/datasets/movielens/20m/).

In [4]:
%%bash
mkdir -p data
cd data
if [ ! -f "ml-20m.zip" ]; then
    echo "Downloading data"
    wget http://files.grouplens.org/datasets/movielens/ml-20m.zip
    unzip ml-20m.zip
fi


In [5]:
!ls ./data

hugeCTR  ml-20m  ml-20m.zip


### Movie lens data preprocessing

In [6]:
from argparse import ArgumentParser
import pandas as pd
import torch
import tqdm

In [7]:
MIN_RATINGS = 20
USER_COLUMN = 'userId'
ITEM_COLUMN = 'movieId'

Since the movie lens data contains only positive examples, let us first define an utility function to generate negative samples.

In [8]:
class _TestNegSampler:
    def __init__(self, train_ratings, nb_users, nb_items, nb_neg):
        self.nb_neg = nb_neg
        self.nb_users = nb_users 
        self.nb_items = nb_items 

        # compute unique ids for quickly created hash set and fast lookup
        ids = (train_ratings[:, 0] * self.nb_items) + train_ratings[:, 1]
        self.set = set(ids)

    def generate(self, batch_size=128*1024):
        users = torch.arange(0, self.nb_users).reshape([1, -1]).repeat([self.nb_neg, 1]).transpose(0, 1).reshape(-1)

        items = [-1] * len(users)

        random_items = torch.LongTensor(batch_size).random_(0, self.nb_items).tolist()
        print('Generating validation negatives...')
        for idx, u in enumerate(tqdm.tqdm(users.tolist())):
            if not random_items:
                random_items = torch.LongTensor(batch_size).random_(0, self.nb_items).tolist()
            j = random_items.pop()
            while u * self.nb_items + j in self.set:
                if not random_items:
                    random_items = torch.LongTensor(batch_size).random_(0, self.nb_items).tolist()
                j = random_items.pop()

            items[idx] = j
        items = torch.LongTensor(items)
        return items

Next, we read the data into a Pandas dataframe, and encode userID and itemID with integers.

In [9]:
df = pd.read_csv('./data/ml-20m/ratings.csv')
print("Filtering out users with less than {} ratings".format(MIN_RATINGS))
grouped = df.groupby(USER_COLUMN)
df = grouped.filter(lambda x: len(x) >= MIN_RATINGS)

print("Mapping original user and item IDs to new sequential IDs")
df[USER_COLUMN], unique_users = pd.factorize(df[USER_COLUMN])
df[ITEM_COLUMN], unique_items = pd.factorize(df[ITEM_COLUMN])

nb_users = len(unique_users)
nb_items = len(unique_items)

print("Number of users: %d\nNumber of items: %d"%(len(unique_users), len(unique_items)))

# Save the mapping to do the inference later on
import pickle
with open('./mappings.pickle', 'wb') as handle:
    pickle.dump({"users": unique_users, "items": unique_items}, handle, protocol=pickle.HIGHEST_PROTOCOL)

Filtering out users with less than 20 ratings
Mapping original user and item IDs to new sequential IDs
Number of users: 138493
Number of items: 26744


Next, we split the data into a train and test set, the last movie each user has recently seen will be used for the test set.

In [10]:
# Need to sort before popping to get last item
df.sort_values(by='timestamp', inplace=True)
    
# clean up data
del df['rating'], df['timestamp']
df = df.drop_duplicates() # assuming it keeps order

# now we have filtered and sorted by time data, we can split test data out
grouped_sorted = df.groupby(USER_COLUMN, group_keys=False)
test_data = grouped_sorted.tail(1).sort_values(by=USER_COLUMN)
# need to pop for each group
train_data = grouped_sorted.apply(lambda x: x.iloc[:-1])

In [11]:
train_data['target']=1
test_data['target']=1
train_data.head()

,userId,movieId,target
20,0,20,1
19,0,19,1
86,0,86,1
61,0,61,1
23,0,23,1


Next, we generate the negative samples for training

In [12]:
sampler = _TestNegSampler(df.values, nb_users, nb_items, 500)  # using 500 negative samples
train_negs = sampler.generate()
train_negs = train_negs.reshape(-1, 500)

sampler = _TestNegSampler(df.values, nb_users, nb_items, 100)  # using 100 negative samples
test_negs = sampler.generate()
test_negs = test_negs.reshape(-1, 100)

  0%|          | 61906/69246500 [00:00<01:51, 619053.21it/s]

Generating validation negatives...


  0%|          | 61238/13849300 [00:00<00:22, 612371.82it/s]

Generating validation negatives...


100%|██████████| 13849300/13849300 [00:19<00:00, 709755.98it/s]


In [13]:
import numpy as np

# generating negative samples for training
train_data_neg = np.zeros((train_negs.shape[0]*train_negs.shape[1],3), dtype=int)
idx = 0
for i in tqdm.tqdm(range(train_negs.shape[0])):
    for j in range(train_negs.shape[1]):
        train_data_neg[idx, 0] = i # user ID
        train_data_neg[idx, 1] = train_negs[i, j] # negative item ID
        idx += 1
    
# generating negative samples for testing
test_data_neg = np.zeros((test_negs.shape[0]*test_negs.shape[1],3), dtype=int)
idx = 0
for i in tqdm.tqdm(range(test_negs.shape[0])):
    for j in range(test_negs.shape[1]):
        test_data_neg[idx, 0] = i
        test_data_neg[idx, 1] = test_negs[i, j]
        idx += 1

100%|██████████| 138493/138493 [01:20<00:00, 1716.78it/s]


In [14]:
train_data_np= np.concatenate([train_data_neg, train_data.values])
np.random.shuffle(train_data_np)

test_data_np= np.concatenate([test_data_neg, test_data.values])
np.random.shuffle(test_data_np)


In [15]:
# HugeCTR expect user ID and item ID to be different, so we use 0 -> nb_users for user IDs and
# nb_users -> nb_users+nb_items for item IDs.
train_data_np[:,1] += nb_users 
test_data_np[:,1] += nb_users 

In [16]:
np.max(train_data_np[:,1])

165236

### Write HugeCTR data files

Next, we will write the data to disk using HugeCTR norm format. 

In [17]:
from ctypes import c_longlong as ll
from ctypes import c_uint
from ctypes import c_float
from ctypes import c_int

def write_hugeCTR_data(huge_ctr_data, filename='huge_ctr_data.dat'):
    print("Writing %d samples"%huge_ctr_data.shape[0])
    with open(filename, 'wb') as f:
        #write header
        f.write(ll(0)) # 0: no error check; 1: check_num
        f.write(ll(huge_ctr_data.shape[0])) # the number of samples in this data file
        f.write(ll(1)) # dimension of label
        f.write(ll(1)) # dimension of dense feature
        f.write(ll(2)) # long long slot_num
        for _ in range(3): f.write(ll(0)) # reserved for future use

        for i in tqdm.tqdm(range(huge_ctr_data.shape[0])):
            f.write(c_float(huge_ctr_data[i,2])) # float label[label_dim];
            f.write(c_float(0)) # dummy dense feature
            f.write(c_int(1)) # slot 1 nnz: user ID
            f.write(c_uint(huge_ctr_data[i,0]))
            f.write(c_int(1)) # slot 2 nnz: item ID
            f.write(c_uint(huge_ctr_data[i,1]))

#### Train data

In [18]:
!rm -rf ./data/hugeCTR
!mkdir ./data/hugeCTR

for i, data_arr in enumerate(np.array_split(train_data_np,10)):
    write_hugeCTR_data(data_arr, filename='./data/hugeCTR/huge_ctr_data_%d.dat'%i)
    
with open('./data/hugeCTR/filelist.txt', 'wt') as f:
    f.write('10\n');
    for i in range(10):
        f.write('./data/hugeCTR/huge_ctr_data_%d.dat\n'%i)

  0%|          | 0/8910827 [00:00<?, ?it/s]

Writing 8910827 samples


  0%|          | 16099/8910827 [00:00<00:55, 160986.32it/s]

Writing 8910827 samples


  0%|          | 14975/8910827 [00:00<00:59, 149749.43it/s]

...

  0%|          | 15908/8910827 [00:00<00:55, 158870.31it/s]

Writing 8910827 samples


  0%|          | 32208/8910827 [00:00<00:56, 156710.59it/s]

Writing 8910827 samples


100%|██████████| 8910827/8910827 [00:51<00:00, 173204.29it/s]


#### Test data

In [19]:
for i, data_arr in enumerate(np.array_split(test_data_np,10)):
    write_hugeCTR_data(data_arr, filename='./data/hugeCTR/test_huge_ctr_data_%d.dat'%i)

  2%|▏         | 32209/1398780 [00:00<00:08, 156935.43it/s]

Writing 1398780 samples


  1%|          | 15753/1398780 [00:00<00:08, 157529.02it/s]

Writing 1398780 samples


  2%|▏         | 32295/1398780 [00:00<00:08, 157409.94it/s]

...

  2%|▏         | 33061/1398779 [00:00<00:08, 163435.77it/s]

Writing 1398779 samples


  2%|▏         | 21363/1398779 [00:01<01:30, 15174.41it/s] 

Writing 1398779 samples


100%|██████████| 1398779/1398779 [00:09<00:00, 154279.37it/s]


In [20]:
with open('./data/hugeCTR/test_filelist.txt', 'wt') as f:
    f.write('10\n');
    for i in range(10):
        f.write('./data/hugeCTR/test_huge_ctr_data_%d.dat\n'%i)

<a id="3"></a>
## 3. HugeCTR DLRM training

In this section, we will train a DLRM network on the augmented movie lens data. First, we write the model config file.

In [16]:
%%writefile dlrm_config.json
{
  "solver": {
    "lr_policy": "fixed",
    "display": 1000,
    "max_iter":50000,
    "gpu": [0],
    "batchsize": 65536,
    "snapshot": 3000,
    "snapshot_prefix": "./hugeCTR_saved_model_DLRM/",
    "eval_interval": 3000,
    "eval_batches": 1000,
    "mixed_precision": 1024,
    "eval_metrics": ["AUC:1.0"]
  },
 
  "optimizer": {
    "type": "SGD",
    "update_type": "Local",
    "sgd_hparam": {
    "learning_rate": 0.1,
    "warmup_steps": 1000,
    "decay_start": 10000,
    "decay_steps": 40000,
    "end_lr": 1e-5
    }
  },


  "layers": [ 
      {
      "name": "data",
      "type": "Data",
      "slot_size_array": [138493 , 26744],
      "slot_size_array_orig": [138493 , 26744],
      "source": "./data/hugeCTR/filelist.txt",
      "eval_source": "./data/hugeCTR/test_filelist.txt",
      "check": "None",
      "cache_eval_data": true,
      "label": {
              "top": "label",
              "label_dim": 1
      },
      "dense": {
              "top": "dense",
              "dense_dim": 1
      },
      "sparse": [
              {
          "top": "data1",
          "type": "LocalizedSlot",
          "max_feature_num_per_sample": 2,
          "max_nnz": 1,
          "slot_num": 2
              }
      ]
    },

    {
      "name": "sparse_embedding1",
      "type": "LocalizedSlotSparseEmbeddingHash",
      "bottom": "data1",
      "top": "sparse_embedding1",
      "sparse_embedding_hparam": {
        "slot_size_array": [138493 , 26744],
        "embedding_vec_size": 64,
        "combiner": 0
      }
    },

    {
      "name": "fc1",
      "type": "FusedInnerProduct",
      "bottom": "dense",
      "top": "fc1",
      "fc_param": {
        "num_output": 64
      }
    },

    {
      "name": "fc2",
      "type": "FusedInnerProduct",
      "bottom": "fc1",
      "top": "fc2",
      "fc_param": {
        "num_output": 128
      }
    },

   
    {
      "name": "fc3",
      "type": "FusedInnerProduct",
      "bottom": "fc2",
      "top": "fc3",
      "fc_param": {
        "num_output": 64
      }
    },

    {
      "name": "interaction1",
      "type": "Interaction",
      "bottom": ["fc3", "sparse_embedding1"],
      "top": "interaction1"
    },

    {
      "name": "fc4",
      "type": "FusedInnerProduct",
      "bottom": "interaction1",
      "top": "fc4",
       "fc_param": {
        "num_output": 1024
      }
    },

    {
      "name": "fc5",
      "type": "FusedInnerProduct",
      "bottom": "fc4",
      "top": "fc5",
      "fc_param": {
        "num_output": 1024
      }
    },

    {
      "name": "fc6",
      "type": "FusedInnerProduct",
      "bottom": "fc5",
      "top": "fc6",
      "fc_param": {
        "num_output": 512
      }
    },

    {
      "name": "fc7",
      "type": "FusedInnerProduct",
      "bottom": "fc6",
      "top": "fc7",
      "fc_param": {
        "num_output": 256
      }
    },

    {
      "name": "fc8",
      "type": "InnerProduct",
      "bottom": "fc7",
      "top": "fc8",
      "fc_param": {
        "num_output": 1
      }
    },
    
    {
      "name": "loss",
      "type": "BinaryCrossEntropyLoss",
      "bottom": ["fc8","label"],
      "top": "loss"
    } 
  ]
}

Overwriting dlrm_config.json


In [17]:
!rm -rf ./hugeCTR_saved_model_DLRM/
!mkdir ./hugeCTR_saved_model_DLRM/

In [18]:
!CUDA_VISIBLE_DEVICES=0 ../build/bin/huge_ctr --train ./dlrm_config.json

[0, init_start, ]
HugeCTR Version: 2.2
Config file: ./dlrm_config.json
[24d23h35m23s][HUGECTR][INFO]: batchsize_eval is not specified using default: 65536
Mixed Precision training with scaler: 1024 is enabled.
[24d23h35m23s][HUGECTR][INFO]: algorithm_search is not specified using default: 1
[24d23h35m23s][HUGECTR][INFO]: Algorithm search: ON
[24d23h35m24s][HUGECTR][INFO]: Peer-to-peer access cannot be fully enabled.
Device 0: Tesla V100-PCIE-32GB
[24d23h35m24s][HUGECTR][INFO]: Initial seed is 4008017228
[24d23h35m24s][HUGECTR][INFO]: num_internal_buffers 1
[24d23h35m24s][HUGECTR][INFO]: num_internal_buffers 1
[24d23h35m24s][HUGECTR][INFO]: max_vocabulary_size_per_gpu_=165237
[24d23h35m24s][HUGECTR][INFO]: All2All Warmup Start
[24d23h35m24s][HUGECTR][INFO]: All2All Warmup End
[24d23h36m52s][HUGECTR][INFO]: gpu0 start to init embedding of slot0 , slot_size=138493, key_offset=0, value_index_offset=0
[24d23h36m52s][HUGECTR][INFO]: gpu0 start to init embedding of slot1 , slot_size=26744, ke

[24d23h46m40s][HUGECTR][INFO]: Iter: 31000 Time(1000 iters): 23.317298s Loss: 0.153621 lr:0.022560
[24d23h46m57s][HUGECTR][INFO]: Iter: 32000 Time(1000 iters): 16.869943s Loss: 0.149449 lr:0.020248
[24d23h47m14s][HUGECTR][INFO]: Iter: 33000 Time(1000 iters): 16.877676s Loss: 0.151620 lr:0.018060
[24d23h47m14s][HUGECTR][INFO]: Rank0: Dump hash table from GPU0
[24d23h47m14s][HUGECTR][INFO]: Rank0: Write hash table <key,value> pairs to file
[24d23h47m14s][HUGECTR][INFO]: Done
[711389, eval_start, 0.66, ]
[24d23h47m20s][HUGECTR][INFO]: Evaluation, AUC: 0.942950
[717705, eval_accuracy, 0.94295, 0.66, 33000, ]
[24d23h47m20s][HUGECTR][INFO]: Eval Time for 1000 iters: 6.315873s
[717705, eval_stop, 0.66, ]
[24d23h47m37s][HUGECTR][INFO]: Iter: 34000 Time(1000 iters): 23.321837s Loss: 0.151781 lr:0.015998
[24d23h47m54s][HUGECTR][INFO]: Iter: 35000 Time(1000 iters): 16.873076s Loss: 0.151561 lr:0.014061
[24d23h48m11s][HUGECTR][INFO]: Iter: 36000 Time(1000 iters): 16.874067s Loss: 0.151250 lr:0.012

<a id="4"></a>
## 4. Answer item similarity with DLRM embedding

In this section, we demonstrate how the output of HugeCTR training can be used to carry out simple inference tasks. Specifically, we will show that the movie embeddings can be used for simple item-to-item similarity queries. Such a simple inference can be used as an efficient candidate generator to generate a small set of cadidates prior to deep learning model re-ranking. 

First, we read the embedding tables and extract the movie embeddings.

In [26]:
import struct 
import pickle
import numpy as np

key_type = 'I32' # {'I64', 'I32'}, default is 'I32'
key_type_map = {"I32": ["I", 4], "I64": ["q", 8]}

embedding_vec_size = 64

HUGE_CTR_VERSION = 2.21 # set HugeCTR version here, 2.2 for v2.2, 2.21 for v2.21

if HUGE_CTR_VERSION <= 2.2:
    each_key_size = key_type_map[key_type][1] + key_type_map[key_type][1] + 4 * embedding_vec_size
else:
    each_key_size = key_type_map[key_type][1] + 8 + 4 * embedding_vec_size

In [39]:
embedding_table = [{},{}]

with open('./hugeCTR_saved_model_DLRM/0_sparse_9000.model', 'rb') as file:
    try:
        while True:
            buffer = file.read(each_key_size)
            if len(buffer) == 0:
                break
            if HUGE_CTR_VERSION <= 2.2:
                key, slot_id = struct.unpack("2" + key_type_map[key_type][0], 
                                             buffer[0: 2*key_type_map[key_type][1]])
                values = struct.unpack(str(embedding_vec_size) + "f", buffer[2*key_type_map[key_type][1]: ])
            else:
                key = struct.unpack(key_type_map[key_type][0], buffer[0 : key_type_map[key_type][1]])[0]
                slot_id = struct.unpack("Q", buffer[key_type_map[key_type][1] : key_type_map[key_type][1] + 8])[0]
                values = struct.unpack(str(embedding_vec_size) + "f", buffer[key_type_map[key_type][1] + 8: ])
            if slot_id==0:
                embedding_table[slot_id][key] = values
            elif slot_id==1:
                embedding_table[slot_id][key - 138493] = values 
            else:
                raise(Exception("Slot ID not found - %d"%slot_id))
            
    except BaseException as error:
        print(error)

In [40]:
item_embedding = np.zeros((26744, embedding_vec_size), dtype='float')
for i in range(len(embedding_table[1])):
    item_embedding[i] = embedding_table[1][i]
    

In [41]:
len(embedding_table[1])

26744

### Answer nearest neighbor queries


In [42]:
from scipy.spatial.distance import cdist

def find_similar_movies(nn_movie_id, item_embedding, k=10, metric="euclidean"):
    #find the top K similar items according to one of the distance metric: cosine or euclidean
    sim = 1-cdist(item_embedding, item_embedding[nn_movie_id].reshape(1, -1), metric=metric)
   
    return sim.squeeze().argsort()[-k:][::-1]

In [43]:
with open('./mappings.pickle', 'rb') as handle:
    movies_mapping = pickle.load(handle)["items"]

nn_to_movies = movies_mapping
movies_to_nn = {}
for i in range(len(movies_mapping)):
    movies_to_nn[movies_mapping[i]] = i

import pandas as pd
movies = pd.read_csv("./data/ml-20m/movies.csv", index_col="movieId")


In [44]:
for movie_ID in range(1,1000):
    try:
        print("Query: ", movies.loc[movie_ID]["title"], movies.loc[movie_ID]["genres"])

        print("Similar movies: ")
        similar_movies = find_similar_movies(movies_to_nn[movie_ID], item_embedding)

        for i in similar_movies:
            print(nn_to_movies[i], movies.loc[nn_to_movies[i]]["title"], movies.loc[nn_to_movies[i]]["genres"])
        print("=================================\n")
    except Exception as e:
        pass

Query:  Toy Story (1995) Adventure|Animation|Children|Comedy|Fantasy
Similar movies: 
1 Toy Story (1995) Adventure|Animation|Children|Comedy|Fantasy
457 Fugitive, The (1993) Thriller
858 Godfather, The (1972) Crime|Drama
588 Aladdin (1992) Adventure|Animation|Children|Comedy|Musical
110 Braveheart (1995) Action|Drama|War
260 Star Wars: Episode IV - A New Hope (1977) Action|Adventure|Sci-Fi
780 Independence Day (a.k.a. ID4) (1996) Action|Adventure|Sci-Fi|Thriller
2571 Matrix, The (1999) Action|Sci-Fi|Thriller
50 Usual Suspects, The (1995) Crime|Mystery|Thriller
527 Schindler's List (1993) Drama|War

Query:  Jumanji (1995) Adventure|Children|Fantasy
Similar movies: 
2 Jumanji (1995) Adventure|Children|Fantasy
2115 Indiana Jones and the Temple of Doom (1984) Action|Adventure|Fantasy
2683 Austin Powers: The Spy Who Shagged Me (1999) Action|Adventure|Comedy
1208 Apocalypse Now (1979) Action|Drama|War
1584 Contact (1997) Drama|Sci-Fi
1080 Monty Python's Life of Brian (1979) Comedy
1258 Shini

30 Shanghai Triad (Yao a yao yao dao waipo qiao) (1995) Crime|Drama
52885 Paprika (Papurika) (2006) Animation|Mystery|Sci-Fi
32031 Robots (2005) Adventure|Animation|Children|Comedy|Fantasy|Sci-Fi|IMAX
49274 Happy Feet (2006) Adventure|Animation|Children|Comedy|IMAX
4345 Anniversary Party, The (2001) Drama
346 Backbeat (1993) Drama|Musical
1085 Old Man and the Sea, The (1958) Adventure|Drama
90746 Adventures of Tintin, The (2011) Action|Animation|Mystery|IMAX
117 Young Poisoner's Handbook, The (1995) Crime|Drama
710 Celtic Pride (1996) Comedy

Query:  Dangerous Minds (1995) Drama
Similar movies: 
31 Dangerous Minds (1995) Drama
5 Father of the Bride Part II (1995) Comedy
3082 World Is Not Enough, The (1999) Action|Adventure|Thriller
170 Hackers (1995) Action|Adventure|Crime|Thriller
35836 40-Year-Old Virgin, The (2005) Comedy|Romance
1884 Fear and Loathing in Las Vegas (1998) Adventure|Comedy|Drama
4370 A.I. Artificial Intelligence (2001) Adventure|Drama|Sci-Fi
610 Heavy Metal (1981) Ac

59 Confessional, The (Confessionnal, Le) (1995) Drama|Mystery
78041 Killers (2010) Action|Comedy
6815 Into the Night (1985) Action|Comedy|Drama|Thriller
90719 J. Edgar (2011) Drama
7826 Secret Life of Walter Mitty, The (1947) Comedy|Romance|Thriller
80864 You Will Meet a Tall Dark Stranger (2010) Comedy|Romance
4633 Parents (1989) Comedy|Drama|Horror
27865 Azumi (2003) Action|Adventure|Drama|Thriller
5651 Incredible Mr. Limpet, The (1964) Animation|Comedy|War
1068 Crossfire (1947) Crime|Film-Noir

Query:  Indian in the Cupboard, The (1995) Adventure|Children|Fantasy
Similar movies: 
60 Indian in the Cupboard, The (1995) Adventure|Children|Fantasy
637 Sgt. Bilko (1996) Comedy
27773 Old Boy (2003) Mystery|Thriller
2642 Superman III (1983) Action|Adventure|Sci-Fi
491 Man Without a Face, The (1993) Drama
47610 Illusionist, The (2006) Drama|Fantasy|Mystery|Romance
4343 Evolution (2001) Comedy|Sci-Fi
2144 Sixteen Candles (1984) Comedy|Romance
3826 Hollow Man (2000) Horror|Sci-Fi|Thriller
748

86 White Squall (1996) Action|Adventure|Drama
3060 Commitments, The (1991) Comedy|Drama|Musical
546 Super Mario Bros. (1993) Action|Adventure|Children|Comedy|Fantasy|Sci-Fi
7371 Dogville (2003) Drama|Mystery|Thriller
76251 Kick-Ass (2010) Action|Comedy
4007 Wall Street (1987) Drama
5049 48 Hrs. (1982) Action|Comedy|Crime|Drama
2118 Dead Zone, The (1983) Thriller
1081 Victor/Victoria (1982) Comedy|Musical|Romance
8910 I Heart Huckabees (2004) Comedy

Query:  Dunston Checks In (1996) Children|Comedy
Similar movies: 
87 Dunston Checks In (1996) Children|Comedy
53129 Mr. Brooks (2007) Crime|Drama|Thriller
52885 Paprika (Papurika) (2006) Animation|Mystery|Sci-Fi
2553 Village of the Damned (1960) Horror|Sci-Fi|Thriller
2500 Jawbreaker (1999) Comedy
7449 Godsend (2004) Drama|Horror|Thriller
6966 Darkman (1990) Action|Crime|Fantasy|Sci-Fi|Thriller
85020 Mechanic, The (2011) Action|Drama|Thriller
1606 Kull the Conqueror (1997) Action|Adventure
2940 Gilda (1946) Drama|Film-Noir|Mystery|Romance



114 Margaret's Museum (1995) Drama
7206 Mon Oncle (My Uncle) (1958) Comedy
4965 Business of Strangers, The (2001) Action|Drama|Thriller
3892 Anatomy (Anatomie) (2000) Horror
4113 Glass Menagerie, The (1987) Drama
49917 When the Levees Broke: A Requiem in Four Acts (2006) Documentary
6872 House of the Dead, The (2003) Action|Horror
119141 The Interview (2014) Action|Comedy
632 Land and Freedom (Tierra y libertad) (1995) Drama|War
52462 Aqua Teen Hunger Force Colon Movie Film for Theaters (2007) Action|Adventure|Animation|Comedy|Fantasy|Mystery|Sci-Fi

Query:  Happiness Is in the Field (Bonheur est dans le pré, Le) (1995) Comedy
Similar movies: 
115 Happiness Is in the Field (Bonheur est dans le pré, Le) (1995) Comedy
65468 Dallas (1950) Western
108149 At Berkeley (2013) Documentary
110763 Made for Each Other (1971) Comedy
103806 90 Minutes (90 minutter) (2012) Drama
98513 Statue of Liberty, The (1985) Documentary
102765 Bush Mama (1979) Drama
977 Moonlight Murder (1936) Mystery
118868 S

...

862 Manny & Lo (1996) Drama
5826 Rio Grande (1950) Romance|Western
27912 Outfoxed: Rupert Murdoch's War on Journalism (2004) Documentary
2634 Mummy, The (1959) Horror
33672 Lords of Dogtown (2005) Action|Comedy|Drama
4599 Great Balls of Fire! (1989) Drama
65868 Repo! The Genetic Opera (2008) Fantasy|Horror|Musical
2035 Blackbeard's Ghost (1968) Children|Comedy
8620 Exterminating Angel, The (Ángel exterminador, El) (1962) Comedy|Drama|Fantasy|Mystery
8039 Support Your Local Sheriff! (1969) Comedy|Western

Query:  Wife, The (1995) Comedy|Drama
Similar movies: 
864 Wife, The (1995) Comedy|Drama
7241 Kanal (1957) Drama|War
27851 Fond Kiss, A (Ae Fond Kiss...) (2004) Drama|Romance
30854 River, The (He liu) (1997) Drama
25757 Jazz Singer, The (1927) Drama|Musical|Romance
3315 Happy Go Lovely (1951) Musical
96314 Celeste and Jesse Forever (Celeste & Jesse Forever) (2012) Comedy|Drama|Romance
53574 TV Set, The (2006) Comedy|Drama
4877 Better Than Sex (2000) Comedy|Romance
25819 Mark of the Vam

893 Mother Night (1996) Drama
3985 Eagle Has Landed, The (1976) Drama|War
5111 Good Son, The (1993) Drama|Thriller
31431 Boogeyman (2005) Drama|Horror|Mystery|Thriller
3975 Lucky Numbers (2000) Comedy|Drama
7340 Just One of the Guys (1985) Comedy
6611 Umberto D. (1952) Drama
52579 Vie en Rose, La (Môme, La) (2007) Drama|Musical
56805 Walk Hard: The Dewey Cox Story (2007) Comedy|Musical
26422 Hair (1979) Comedy|Drama|Musical

Query:  Liebelei (1933) Romance
Similar movies: 
894 Liebelei (1933) Romance
96004 Dragon Ball Z: The History of Trunks (Doragon bôru Z: Zetsubô e no hankô!! Nokosareta chô senshi - Gohan to Torankusu) (1993) Action|Adventure|Animation
87302 Red White & Blue (2010) Drama|Mystery|Romance
113565 Sacrament, The (2013) Horror|Thriller
80761 Crimson Wing: Mystery of the Flamingos, The (2008) Documentary
101402 Riders of the Purple Sage  (1996) Drama|Romance|Western
80311 Come On, Rangers (1938) Romance|Western
48458 Sidewalks of London (St. Martin's Lane) (1938) Comedy


922 Sunset Blvd. (a.k.a. Sunset Boulevard) (1950) Drama|Film-Noir|Romance
535 Short Cuts (1993) Drama
866 Bound (1996) Crime|Drama|Romance|Thriller
33004 Hitchhiker's Guide to the Galaxy, The (2005) Adventure|Comedy|Sci-Fi
207 Walk in the Clouds, A (1995) Drama|Romance
4776 Training Day (2001) Crime|Drama|Thriller
3424 Do the Right Thing (1989) Drama
1616 Peacemaker, The (1997) Action|Thriller|War
3285 Beach, The (2000) Adventure|Drama
2686 Red Violin, The (Violon rouge, Le) (1998) Drama|Mystery

Query:  Citizen Kane (1941) Drama|Mystery
Similar movies: 
923 Citizen Kane (1941) Drama|Mystery
79132 Inception (2010) Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX
6365 Matrix Reloaded, The (2003) Action|Adventure|Sci-Fi|Thriller|IMAX
1127 Abyss, The (1989) Action|Adventure|Sci-Fi|Thriller
1573 Face/Off (1997) Action|Crime|Drama|Thriller
1394 Raising Arizona (1987) Comedy
1610 Hunt for Red October, The (1990) Action|Adventure|Thriller
11 American President, The (1995) Comedy|Drama|Romance


952 Around the World in 80 Days (1956) Adventure|Comedy
2558 Forces of Nature (1999) Comedy|Romance
7387 Dawn of the Dead (1978) Action|Drama|Horror
2106 Swing Kids (1993) Drama|War
53121 Shrek the Third (2007) Adventure|Animation|Children|Comedy|Fantasy
971 Cat on a Hot Tin Roof (1958) Drama
1422 Murder at 1600 (1997) Crime|Drama|Mystery|Thriller
3987 Bounce (2000) Drama|Romance
1010 Love Bug, The (1969) Children|Comedy
68554 Angels & Demons (2009) Crime|Drama|Mystery|Thriller

Query:  It's a Wonderful Life (1946) Drama|Fantasy|Romance
Similar movies: 
953 It's a Wonderful Life (1946) Drama|Fantasy|Romance
6365 Matrix Reloaded, The (2003) Action|Adventure|Sci-Fi|Thriller|IMAX
1148 Wallace & Gromit: The Wrong Trousers (1993) Animation|Children|Comedy|Crime
3949 Requiem for a Dream (2000) Drama
1090 Platoon (1986) Drama|War
1396 Sneakers (1992) Action|Comedy|Crime|Drama|Sci-Fi
44191 V for Vendetta (2006) Action|Sci-Fi|Thriller|IMAX
1997 Exorcist, The (1973) Horror|Mystery
1777 Wedding S

982 Picnic (1955) Drama
8610 All of Me (1984) Comedy|Fantasy
7117 Leprechaun (1993) Comedy|Horror
5927 Best Little Whorehouse in Texas, The (1982) Comedy|Drama|Musical|Romance
102123 This Is the End (2013) Action|Comedy
4636 Punisher, The (1989) Action
6790 Avalon (2001) Drama|Fantasy|Sci-Fi
49824 Dreamgirls (2006) Drama|Musical
201 Three Wishes (1995) Drama|Fantasy
75985 Repo Men (2010) Action|Sci-Fi|Thriller

Query:  Madagascar Skin (1995) Romance
Similar movies: 
983 Madagascar Skin (1995) Romance
91223 Constant Nymph, The (1943) Drama|Romance
113970 Mi Amigo Hugo (2014) Documentary
58661 Dead Awake (2001) Action|Thriller
105794 Another Nine & a Half Weeks (Love in Paris) (9 1/2 Weeks II) (Another 9 1/2 Weeks) (1997) Drama|Romance
6028 Easy Come, Easy Go (1967) Adventure|Musical
92600 As Luck Would Have It (Chispa de la vida, La) (2011) Drama
7768 Guncrazy (1992) Crime|Drama|Thriller
119086 A Follower for Emily (1974) Drama
109987 Love Simple (2009) Comedy|Romance

Query:  Pompatus 